# Demo ShiftAmountActivity
This activity uses a processor to transfer a specified number of objects from an origin resource, which must have a container, to a destination resource, which also must have a container.

Shift Amount checks the possible amount of objects which can be transferred, based on the number of objects available in the origin, the number of objects which can be stored in the destination and the number of objects requested to be transferred. If the number of actually to be transferred objects is zero than an exception is raised. These cases have to be prevented by using appropriate events.

In [1]:
import datetime, time
import simpy

import shapely.geometry

import pandas as pd
import openclsim.core as core
import openclsim.model as model

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of Sites

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "geometry": location_from_site,  # The coordinates of the project site
    "capacity": 10,  # The capacity of the site
    "level": 2,
}  # The actual volume of the site

# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "geometry": location_to_site,  # The coordinates of the project site
    "capacity": 10,  # The capacity of the site
    "level": 0,
}  # The actual volume of the site (empty of course)

# The two objects used for the simulation
from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

## Creation of Vessel

In [3]:
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

def compute_v_provider(v_empty, v_full):
    return lambda x: 10

data_hopper = {
    "env": my_env,
    "name": "Hopper 01",
    "geometry": location_from_site,
    "loading_rate": 1,
    "unloading_rate": 1, 
    "capacity": 5,
    "compute_v": compute_v_provider(5, 4.5),
}

hopper = TransportProcessingResource(**data_hopper)

## Definition of Shift Amount activity

In [4]:
shift_amount_activity_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer MP",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5b",  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 100,
    "duration": 10,
}
activity = model.ShiftAmountActivity(**shift_amount_activity_data)

## Run simulation

In [5]:
my_env.run()

In [6]:
log_df = pd.DataFrame(activity.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,Transfer MP,START,1970-01-01 00:00:00,2,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
1,Transfer MP,STOP,1970-01-01 00:00:10,2,6dbbbdf7-4589-11e9-bf3b-b469212bff5b


The resulting levels ob objects in the hopper and the from_site are requested below. 